### Function collection
* Collection of functions which Julian initially computed for the wetland classification

In [ ]:
#Version I: annual stats
#Version II: including a seasonal filter (as in the LUCAS analysis)

def calculate_predictors(tile): 

    roi = tiles.filterMetadata(tilevar, 'equals', tile)
   
    left = ee.Number(1)
    right = ee.Number(1)
    
    years = ee.List.sequence(1984, 2017)
    
    def year_predictors(year):
        
        start = ee.Date.fromYMD(ee.Number(year), 1, 1)
        end = ee.Date.fromYMD(ee.Number(year), 12, 31)

        start_window = ee.Date.fromYMD(ee.Number(year).subtract(left), 1, 1)
        end_window = ee.Date.fromYMD(ee.Number(year).add(right), 12, 31)

        landsat_data = landsat_collect(roi, start_window, end_window)
        spectral_indices = landsat_indices(landsat_data, ["NBR", "EVI", "wetness", "brightness", "greenness"])
        percentiles = spectral_indices.reduce(ee.Reducer.percentile([5,25,50,75,95]))
        predictors = percentiles.addBands(water).float()

        return predictors.set('system:time_start', start.millis())
    
    annual_list = years.map(year_predictors)
                            
    return ee.ImageCollection(annual_list)

def calculate_predictors_seasonal(tile, year): 

    roi = tiles.filterMetadata(tilevar, 'equals', tile)

    def calculate_filter(seasonal_filter):

        start = ee.Date.fromYMD(ee.Number(year).subtract(ee.Number(1)), 1, 1)
        end = ee.Date.fromYMD(ee.Number(year).add(ee.Number(1)), 12, 31)

        # calculate metrics
        landsat_data = landsat_collect(roi, start, end, seasonal_filter).select(['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2'])
    
        percentiles = ee.Reducer.percentile([5,25,50,75,95]).unweighted()
        minmax = ee.Reducer.minMax().unweighted()
        mean = ee.Reducer.mean().unweighted()
        sd = ee.Reducer.stdDev().unweighted()
        metrics_summary = percentiles.combine(minmax, sharedInputs = True).combine(mean, sharedInputs = True).combine(sd, sharedInputs= True)
        
        clear_count = landsat_data.select('B').count().rename('clear_count')
        
        predictors = landsat_data.reduce(metrics_summary).addBands(clear_count)

        return predictors.float()
    
    # calculate predictors for target seasonal window
    multiband =  calculate_filter(ee.Filter.dayOfYear(135, 258)) # May 15th - September 15th 

    bands_sorted = multiband.bandNames().sort()
    
    return multiband.select(bands_sorted)



In [ ]:
def landsat_indices (landsat, indices):
    
    # helper functions to add spectral indices to collections

    def add_NDVI (image):
        ndvi = image.normalizedDifference(['NIR', 'R']).rename('NDVI')
        return image.addBands(ndvi)

    def add_NBR (image):
        nbr = image.normalizedDifference(['NIR', 'SWIR2']).rename('NBR')
        return image.addBands(nbr)

    def add_NBR2 (image):
        nbr2 = image.normalizedDifference(['SWIR1', 'SWIR2']).rename('NBR2')
        return image.addBands(nbr2)

    def add_NDMI (image):
        ndvi = image.normalizedDifference(['NIR', 'SWIR1']).rename('NDMI')
        return image.addBands(ndvi)

    def add_EVI (image):
        evi = image.expression('2.5 * ((NIR - R) / (NIR + 6 * R - 7.5 * B + 1))', {'NIR': image.select('NIR'),'R': image.select('R'),'B': image.select('B')}).rename('EVI')
        return image.addBands(evi)

    def add_SAVI (image): 
        savi = image.expression('((NIR - R) / (NIR + R + 0.5 )) * (1.5)', {'NIR': image.select('NIR'),'R': image.select('R')}).rename('SAVI')
        return image.addBands(savi)
    
    def add_MSAVI (image): 
        msavi = image.expression('(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - R)) ) / 2', {'NIR': image.select('NIR'),'R': image.select('R')}).rename('MSAVI')
        return image.addBands(msavi)

    def add_TC (image):
        
        img = image.select(['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2'])
        
        # coefficients for Landsat surface reflectance (Crist 1985)
        brightness_c= ee.Image([0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863])
        greenness_c= ee.Image([-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800])
        wetness_c= ee.Image([0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572])
        fourth_c= ee.Image([-0.8242, 0.0849, 0.4392, -0.0580, 0.2012, -0.2768])
        fifth_c= ee.Image([-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085])
        sixth_c= ee.Image([0.1084, -0.9022, 0.4120, 0.0573, -0.0251, 0.0238])

        brightness = img.multiply(brightness_c)
        greenness = img.multiply(greenness_c)
        wetness = img.multiply(wetness_c)
        fourth = img.multiply(fourth_c)
        fifth = img.multiply(fifth_c)
        sixth = img.multiply(sixth_c)

        brightness = brightness.reduce(ee.call('Reducer.sum'))
        greenness = greenness.reduce(ee.call('Reducer.sum'))
        wetness = wetness.reduce(ee.call('Reducer.sum'))
        fourth = fourth.reduce(ee.call('Reducer.sum'))
        fifth = fifth.reduce(ee.call('Reducer.sum'))
        sixth = sixth.reduce(ee.call('Reducer.sum'))

        tasseled_cap = ee.Image(brightness).addBands(greenness).addBands(wetness).addBands(fourth).addBands(fifth).addBands(sixth).rename(['brightness','greenness','wetness','fourth','fifth','sixth'])

        return image.addBands(tasseled_cap)


    out = landsat.map(add_NDVI).map(add_NBR).map(add_NBR2).map(add_NDMI).map(add_EVI).map(add_SAVI).map(add_MSAVI).map(add_TC).select(indices)

    return out
# Additional waterness index based on the pixel qa-band from Landsat
def waterness (landsat):
    
    def addWatermask(image):
        qa = image.select('pixel_qa').rename('watermask')
        return image.addBands(qa.bitwiseAnd(4).neq(0))
    
    def addCloudmask(image):
        qa = image.select('pixel_qa').rename('cloudmask')
        return image.addBands(qa.bitwiseAnd(40).neq(0))
    
    masks = landsat.map(addWatermask).map(addCloudmask).select(['watermask', 'cloudmask'])
    masks_sum = masks.sum()
    
    nobs = landsat.select('pixel_qa').count().rename('count')
    water_sum = masks_sum.select('watermask').rename('water_sum')
    cloud_sum = masks_sum.select('cloudmask').rename('cloud_sum')
    clear_obs = nobs.subtract(cloud_sum).rename('clear_obs')
    waterness = water_sum.divide(clear_obs).rename('waterness')
    
    return waterness